<a href="https://colab.research.google.com/github/M-MSilva/Predict-NBA-player-Points-End-to-end-Project/blob/master/Jupyter_Notebook_Projects/Web_Scraping_NBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping nbastuffer

## 1 - Requirements

In [ ]:
import requests #To make the request to the site
from bs4 import BeautifulSoup as Bfs #to collect data from an HTML or XML
import pandas as pd #To organize and rearrange the tables
import numpy as np #To manipulate lists

##  2 - Get the link of stats of the players

In this part of the code we collect the links from the tables, which have information about the athletes and store them in a list.

In [ ]:
def getLinks(): 
  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

  #collect all links
  links=[]

  #get the page url
  URL = 'https://www.nbastuffer.com/player-stats/'

  #access the site
  issue = requests.get(URL, headers=headers)

  #extract the information
  Web = Bfs(issue.content, 'html.parser')

  #We look for tags of interest
  players = Web.findAll('div', attrs={'class': 'entry-wrap'})


  for item in players:
    atag = item.a #we use the link tag
    link_aux = atag.get('href')
    if link_aux == 'https://www.nbastuffer.com/2017-nba-playoffs-player-stats/':
        break  
    if link_aux:
      links.append(link_aux)#and we put on the list
  return(links)

## 3 - Search athletes automatically and avoid errors

First, we search the table for the tags that give us your information. We use try, except in cases where we can't find the athlete's information on the site, and when that happens, if it's not points, we assign an empty string to the variable.

In [ ]:
def search_auto(athlet_dict,athletes):

  #we count the maximum number of players in the table
  number_of_rows = len(athletes.findAll(lambda tag: tag.name == 'tr')) + 1

  #we take all the lines we are interested in
  for i in range(2,number_of_rows):
    if(i%2==0):
      row = 'row-{}'.format(i) + ' even'
    else:
      row = 'row-{}'.format(i) + ' odd'

    #if we don't find the line we want, we exit the loop
    aux = athletes.find('tr', attrs={'class': row})
    if(aux == 'None'):
      break
  
    #as the player's score is the most important variable, if we don't find such a tag we exit the function
    try:
      athlet_dict['PPG'].append(aux.find('td', attrs={'class': 'column-19'}).text)
    except AttributeError:
      return

    #for all tags we look for them, and if we don't find them we put empty in the string
    try:
      athlet_dict['TEAM'].append(aux.find('td', attrs={'class': 'column-3'}).text)
    except AttributeError:
      athlet_dict['TEAM'].append('')

    try:
      athlet_dict['POS'].append(aux.find('td', attrs={'class': 'column-4'}).text)
    except AttributeError:
      athlet_dict['POS'].append('')

    
    try:
      athlet_dict['AGE'].append(aux.find('td', attrs={'class': 'column-5'}).text)
    except AttributeError:
      athlet_dict['AGE'].append('')

    try:
      athlet_dict['GP'].append(aux.find('td', attrs={'class': 'column-6'}).text)
    except AttributeError:
      athlet_dict['GP'].append('')

    try:
      athlet_dict['MPG'].append(aux.find('td', attrs={'class': 'column-7'}).text)
    except AttributeError:
      athlet_dict['MPG'].append('')
    
    try:
      athlet_dict['MIN%'].append(aux.find('td', attrs={'class': 'column-8'}).text)
    except AttributeError:
      athlet_dict['MIN%'].append('')

    try:
      athlet_dict['USG%'].append(aux.find('td', attrs={'class': 'column-9'}).text)
    except AttributeError:
      athlet_dict['USG%'].append('')

    try:
      athlet_dict['TO%'].append(aux.find('td', attrs={'class': 'column-10'}).text)
    except AttributeError:
      athlet_dict['TO%'].append('')
  
    try:
      athlet_dict['FTA'].append(aux.find('td', attrs={'class': 'column-11'}).text)
    except AttributeError:
      athlet_dict['FTA'].append('')


    try:
      athlet_dict['FT%'].append(aux.find('td', attrs={'class': 'column-12'}).text)
    except AttributeError:
      athlet_dict['FT%'].append('')

    try:
      athlet_dict['2PA'].append(aux.find('td', attrs={'class': 'column-13'}).text)
    except AttributeError:
      athlet_dict['2PA'].append('')

    try:
      athlet_dict['3PA'].append(aux.find('td', attrs={'class': 'column-15'}).text)
    except AttributeError:
      athlet_dict['3PA'].append('')

    try:
      athlet_dict['FULL NAME'].append(aux.find('td', attrs={'class': 'column-2'}).text)
    except AttributeError:
      athlet_dict['FULL NAME'].append('')

    try:
      athlet_dict['RPG'].append(aux.find('td', attrs={'class': 'column-20'}).text)
    except AttributeError:
      athlet_dict['RPG'].append('')

    try:
      athlet_dict['TRB%'].append(aux.find('td', attrs={'class': 'column-21'}).text)
    except AttributeError:
      athlet_dict['TRB%'].append('')

    try:
      athlet_dict['APG'].append(aux.find('td', attrs={'class': 'column-22'}).text)
    except AttributeError:
      athlet_dict['APG'].append('')


    try:
      athlet_dict['SPG'].append(aux.find('td', attrs={'class': 'column-24'}).text)
    except AttributeError:
      athlet_dict['SPG'].append('')


    try:
      athlet_dict['BPG'].append(aux.find('td', attrs={'class': 'column-25'}).text)
    except AttributeError:
      athlet_dict['BPG'].append('')

    try:
      athlet_dict['TOPG'].append(aux.find('td', attrs={'class': 'column-26'}).text)
    except AttributeError:
      athlet_dict['TOPG'].append('')


  return athlet_dict

##  4 - Collect all data

We do the same thing as before with the links of the tables, only now we get all the precious information of the athlete in each table that is on the site.

In [ ]:
from pandas import DataFrame
def CollectAlldata():

  #first we get all links,
  links = getLinks()

  #we create a dictionary with several lists for each column of the table that we will collect.
  player_dict = {'FULL NAME':[], 'TEAM':[],'POS':[],'AGE':[],'GP':[],'MPG':[],'MIN%':[],'USG%':[]
               ,'TO%':[],'FTA':[],'FT%':[],'2PA':[],'3PA':[],'PPG':[],'RPG':[],'TRB%':[],'APG':[]
               ,'SPG':[],'BPG':[],'TOPG':[]}

  #Default website header.
  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
  
  #we travel the links in the list
  for intem_links in links:
  
    #access the site
    issue = requests.get(intem_links, headers=headers)

    #extract the information
    Web = Bfs(issue.content, 'html.parser')


    #we select the tag that contains all the information we need
    athletes = Web.find('div', attrs={'class': 'tablepress-scroll-wrapper'})
    
    #if everything is correct we collect all the tags we insert them in a dictionary, and finally in a dataframe created
    if athletes:
      result = search_auto(player_dict,athletes)
      DataFrame = pd.DataFrame(result)
  
  DataFrame.to_csv('NBA_Athletes.csv',line_terminator='',encoding='utf-8')

In [ ]:
if __name__ == "__main__":
  CollectAlldata()